# Pytest

So far we have seen how to run tests using `pytest`. In this section we will learn how to write tests and take advantage of the powerful [`pytest.mark`](https://docs.pytest.org/en/latest/mark.html) to make these tests more useful and informative.

## Skipping tests

In the previous section we saw that `pytest` could flag tests as `SKIPPED` and report information as to why this was the case. Let's look at one of the test functions to see how this was achieved.

```python
@pytest.mark.skip(reason="Not yet implemented.")
def test_div():
    """ Test the div function. """
    assert div(9, 2) == approx(4.5)
```

Here we have written a test for an, as yet, unimplemented function `div` that divides one number by another and returns the result. For a simple function like this the expected output is obvious so it's easy to write a test before the function is even implemented. We are asserting what the _output_ of the function should be, not how it should be _implemented_.

Writing tests before implementing functionality is often good practice and is referred [_test-driven development_](https://en.wikipedia.org/wiki/Test-driven_development). Writing tests first can help to better structure your code. Once a test is written you should write the minimum functionality that makes the test pass, then add more tests, and refine.

We have marked the test to be skipped by using `@pytest.mark.skip` with a reason given in the parentheses. Don't worry about this funny syntax. It is an example of what's known as a _function decorator_. We are _wrapping_ our test function inside another function called `skip`.

As an example:

In [20]:
def make_big(func):
    print("Big ", end='')
    func()
    
def cat():
    print("cat")
    
make_big(cat)

@make_big
def dog():
    print("dog")

Big cat
Big dog


Let's look at another reason why we might want to skip a test.

```python
@pytest.mark.skipif(sys.platform != 'win32', reason="Only runs on windows.")
def test_BSoD():
    blueScreenOfDeath()
```

Here we the test is marked with a _conditional skip_. The test will only be run if the host operating system is Windows. Adding conditional skips like this allows your test suite to be robust and portable.

## Parametrizing tests

As we have already seen, it is usually desirable to run a test for a range of different input parameters. With `pytest` it is easy to parametrize our test functions.

```python
@pytest.mark.parametrize("x", [1, 2])
@pytest.mark.parametrize("y", [3, 4])
def test_mul(x, y):
    """ Test the mul function. """
    assert mul(x, y) == mul(y, x)
```

Here the function `test_mul` is parametrized with two parameters, `x` and `y`. By marking the test in this manner it will be executed over using all possible parameter pairs `(x, y)`, i.e. `(1, 3), (1, 4), (2, 3), (2, 4)`.

Tests can also be parametrized in a different way.


```python
@pytest.mark.parametrize("x, y, expected",
                        [(1, 2, -1),
                         (7, 3,  4),
                         (21, 58, -37)])
def test_sub(x, y, expected):
    """ Test the sub function. """
    assert sub(x, y) == -sub(y, x) == expected
```

Here we are passing a list containing different parameter sets, with the names of the parameters matched against the arguments of the test function. Each set of parameters contains the two values to be tested, `x` and `y`, as well as the `expected` outcome of the test. This allows the use of a single `assert` statement in the body of the test function. Can you think why having a single assertion is a good thing?

## Expected failures

By using marks we can also indicate that we expect a particular test to fail.

```python
@pytest.mark.xfail(reason="Broken code. Working on a fix.")
def test_broken():
    broken()
```

This is good practice. Rather than hiding tests for our buggy code, we are acknowledging that we are aware of the problem and are working on a fix. The user can query the expected failures and see the reasons for their inclusion. Once bugs have been fixed it is important to keep the tests as part of your codebase. That way you'll know whenever a future change reintroduces a bug that was previously fixed.

We can also use expected failures to test for conditons _not_ being met. As an example, recall the infamous `greaterThan` function.

```python
@pytest.mark.parametrize("x, y",
                        [(2, 1),
                         (9, 4),
                         pytest.param(3, 7, marks=pytest.mark.xfail(strict=False)),
                         pytest.param(6, 11, marks=pytest.mark.xfail(strict=True))])
def test_greaterThan(x, y):
    """ Test the greaterThan function. """
    assert greaterThan(x, y)
```

Here we pass four sets of parameters to the test function. The first two check test for cases where the condition should be _true_, e.g. we know that 2 is greater than 1. In contrast, the final two parameter sets check for cases where the condition should be _false_, i.e. 3 isn't greater than 7. Here we have passed an additional parameter that indicates that we expect the last two cases to be failures.

This allows us to use a single function to perform both types of test (checking that a test both passes and fails when we expect it to). Otherwise we would have needed to use mutiple `assert` statements, or two test functions.

The `xfail` mark can also take an addition keyword parameter, `strict`. This indicates whether expected failures that pass, an `XPASS`, should be classed as a failure.

## Custom attributes

It's possible to mark test functions with any attribute you like. For example:

```python
@pytest.mark.slow
def test_bigSum():
    """ Test the bigSum function. """
    assert bigSum() == 125000000250000000
```

Here we have marked the `test_bigSum` function with the attribute `slow` in order to indicate that it takes a while to run. From the command line it is possible to run or skip tests with a particular mark.

```bash
pytest mymodule -m "slow"        # only run the slow tests
pytest mymodule -m "not slow"    # skips the slow tests
```

# Exercises

Here you'll be modifying the files `mymodule/mymodule.py` and `mymodule/test/test_mymath.py`.

__Exercise 1:__ Pass `strict=True` to both `xfail` marks used in the `test_greaterThan` function. Re-run `pytest` to see how the output has changed.

__Exercise 2:__ Fix the bugs in `mymodule.py` (specifically in the `add` and `greaterThan` functions). Re-run `pytest` and verify that all of the tests pass.

__Exercise 3:__ Parametrize the `test_add` function so that it can work with a single assert statement. Also add a parameter that checks for an expected failure. Verify your work by running `pytest`.

__Exercise 4:__ Add a mark to the `tes_mul` function to indicate that it is `critical`. Run `pytest` only for this `critical` test.